<a href="https://colab.research.google.com/github/Rookey4ever/-Coursera_Capstone/blob/master/Capstone_Project_Battle_of_Neighborhoods_Cbusipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DATA PREP**

The idea is to read data from google sheets into a pandas dataframe. The data is Latitude and Longitude values for different neighborhoods in Columbus, Ohio

In [160]:
!pip install --upgrade -q gspread
!pip install gspread-dataframe

Importing Dependencies to read data from google sheets and general data analysis

In [0]:
import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc= gspread.authorize(GoogleCredentials.get_application_default())

Specify the path of the goolgle sheet

In [0]:
sheet1= gc.open_by_url('https://docs.google.com/spreadsheets/d/1coG3PJnw_VysfXCu_6ZG2pwYeCI1-HkcW5Sl70bERzE/edit#gid=0')

The worksheet is named **cbus**

In [0]:
ws=sheet1.worksheet('cbus')



Reading the worksheet as a pandas dataframe

In [0]:
cbus_df=get_as_dataframe(ws)



In [44]:
cbus_df.head()



,Neighborhood,Lat,Long
0,Bexley,39.969238,-82.936867
1,Brewery District,39.951569,-82.999921
2,Canal Winchester,39.842819,-82.805649
3,Clintonville,44.620739,-88.761971
4,Downtown Columbus,39.962703,-82.997899


The next step is to visualize these neighbourhoods. For this we need to install folium.

In [45]:
!pip install folium
import folium


In [0]:
cbus_map= folium.Map(location=[39.9612, -82.9988], zoom_start=11)
# instantiate a feature group for the incidents in the dataframe
cbus = folium.map.FeatureGroup()

In [47]:


# loop through the 38 neighbourhoods and add each to the cbus feature group
for lat, lng, in zip(cbus_df.Lat, cbus_df.Long):
    cbus.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=8, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

# add cbus to map
cbus_map.add_child(cbus)

**Leverage 4 Square api to to explore popular venues in each neighbourhood**

**Importing Dependencies**

In [0]:
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!pip install geopy
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


In [50]:
CLIENT_ID = 'ESH5EKWLJOCGFF2BZK4SBUYDBDQRZZW1VPSWBCG003P1T2RO' # your Foursquare ID
CLIENT_SECRET = 'XRARIH3DKIDEDMCJY1AWRIDOJHUI1QMPZZQOQP0DCNIUGD3F' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ESH5EKWLJOCGFF2BZK4SBUYDBDQRZZW1VPSWBCG003P1T2RO
CLIENT_SECRET:XRARIH3DKIDEDMCJY1AWRIDOJHUI1QMPZZQOQP0DCNIUGD3F


Explore the neighbourhoods for popular venues

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
cbus_venues = getNearbyVenues(names=cbus_df['Neighborhood'],
                                   latitudes=cbus_df['Lat'],
                                   longitudes=cbus_df['Long']
                                  )

Bexley
Brewery District
Canal Winchester
Clintonville
Downtown Columbus
Dublin
Eastmoor
Franklin Park
Franklinton
Gahanna
German Village
Grandview Heights
Grove City
Groveport
Harrison West
Hiliard
Hilltop
Italian Village
King Lincoln
Linden
Merion Village
Minerva Park
New Albany
North Linden
Northland
Old Towne East
Reynoldsburg
Short North
South Side
University District
Upper Arlington
Victorian Village
Weinland Park
Westerville
Westgate
Whitehall
Woodland Park
Worthington


In [64]:
print(cbus_venues.shape)
cbus_venues.head()

(525, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bexley,39.969238,-82.936867,Commonwealth Park,39.972574,-82.937848,Park
1,Bexley,39.969238,-82.936867,Wolf Park,39.969814,-82.931324,Park
2,Brewery District,39.951569,-82.999921,Arepazo Tapas Bar Grill,39.951473,-82.998798,Latin American Restaurant
3,Brewery District,39.951569,-82.999921,Shadowbox Live,39.951449,-83.001091,Theater
4,Brewery District,39.951569,-82.999921,Rockmill Tavern,39.951322,-83.000425,Brewery


Let's check how many venues were returned for each neighborhood

In [65]:
cbus_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bexley,2,2,2,2,2,2
Brewery District,30,30,30,30,30,30
Canal Winchester,12,12,12,12,12,12
Clintonville,8,8,8,8,8,8
Downtown Columbus,30,30,30,30,30,30
Dublin,29,29,29,29,29,29
Eastmoor,8,8,8,8,8,8
Franklin Park,7,7,7,7,7,7
Franklinton,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [67]:
print('There are {} uniques categories.'.format(len(cbus_venues['Venue Category'].unique())))



There are 160 uniques categories.


**Analyze Each Neighborhood**

In [99]:
# one hot encoding
cbus_onehot = pd.get_dummies(cbus_venues[['Venue Category']], prefix="", prefix_sep="")



#cbus_onehot.insert(loc=0, column='Neighborhood', value=cbus_venues['Neighborhood'])
#add neighborhood column back to dataframe
cbus_onehot['Neighborhood'] = cbus_venues['Neighborhood'] 
cbus_onehot.columns.get_loc("Neighborhood")
# move neighborhood column to the first column
#fixed_columns = [cbus_onehot.columns[-1]] + list(cbus_onehot.columns[:-1])
#cbus_onehot = cbus_onehot[fixed_columns]
#cbus_onehot











105

In [100]:
# move neighborhood column to the first column
fixed_columns = [cbus_onehot.columns[105]] + list(cbus_onehot.columns[:104]) + list(cbus_onehot.columns[106:])
cbus_onehot = cbus_onehot[fixed_columns]
cbus_onehot.head()

,Neighborhood,ATM,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Capitol Building,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,...,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shipping Store,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Bexley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bexley,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Brewery District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brewery District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Brewery District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [103]:
cbus_onehot.shape

(525, 159)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [104]:
cbus_grouped = cbus_onehot.groupby('Neighborhood').mean().reset_index()
cbus_grouped

,Neighborhood,ATM,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Capitol Building,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,...,Print Shop,Pub,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shipping Store,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Bexley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brewery District,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.033333,0.066667,0.000000,0.0,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.100000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Canal Winchester,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Clintonville,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downtown Columbus,0.000000,0.033333,0.00000

Getting Top 10 Venues by neighbourhood

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cbus_grouped['Neighborhood']

for ind in np.arange(cbus_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cbus_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bexley,Park,Yoga Studio,Department Store,Farmers Market,Fabric Shop,Event Space,Donut Shop,Dive Bar,Discount Store,Diner
1,Brewery District,Coffee Shop,Bakery,Bar,American Restaurant,Brewery,Theater,Beer Bar,Breakfast Spot,Restaurant,Italian Restaurant
2,Canal Winchester,Bank,American Restaurant,Pool,Brewery,Snack Place,Café,Bakery,Fast Food Restaurant,Historic Site,History Museum
3,Clintonville,Automotive Shop,Video Store,Sandwich Place,Bar,Fast Food Restaurant,Coffee Shop,Convenience Store,American Restaurant,Convention Center,Farmers Market
4,Downtown Columbus,Coffee Shop,Hotel,Breakfast Spot,Theater,Italian Restaurant,Event Space,Latin American Restaurant,Salad Place,Business Service,Irish Pub
5,Dublin,Pizza Place,Café,Italian Restaurant,Yoga Studio,Brewery,Ice Cream Shop,Steakhouse,Irish Pub,Southern / Soul Food Restaurant,Boutique
6,Eastmoor,Fast Food Restaurant,Chinese Restaurant,Discount Store,Mexican Restaurant,ATM,Gas Station,Convention Center,Cosmetics Shop,Cuban Restaurant,Dance Studio
7,Franklin Park,Park,Construction & Landscaping,Fried Chicken Joint,Baseball Field,Tennis Court,Athletics & Sports,Discount Store,Farmers Market,Fabric Shop,Event Space
8,Franklinton,Gas Station,Diner,Pizza Place,Food Service,Discount Store,Furniture / Home Store,Coffee Shop,Cuban Restaurant,Dance Studio,Deli / Bodega
9,Gahanna,Ice Cream Shop,Breakfast Spot,Bar,Clothing Store,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Cajun / Creole Restaurant,Memorial Site,Mexican Restaurant


## Cluster Neighborhoods

In [153]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

cbus_grouped_clustering = cbus_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbus_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [154]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cbus_merged = cbus_df

# merge cbus_grouped with cbus_data to add latitude/longitude for each neighborhood
cbus_merged = cbus_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cbus_merged.head() # check the last columns!

,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bexley,39.969238,-82.936867,1.0,Park,Yoga Studio,Department Store,Farmers Market,Fabric Shop,Event Space,Donut Shop,Dive Bar,Discount Store,Diner
1,Brewery District,39.951569,-82.999921,0.0,Coffee Shop,Bakery,Bar,American Restaurant,Brewery,Theater,Beer Bar,Breakfast Spot,Restaurant,Italian Restaurant
2,Canal Winchester,39.842819,-82.805649,0.0,Bank,American Restaurant,Pool,Brewery,Snack Place,Café,Bakery,Fast Food Restaurant,Historic Site,History Museum
3,Clintonville,44.620739,-88.761971,0.0,Automotive Shop,Video Store,Sandwich Place,Bar,Fast Food Restaurant,Coffee Shop,Convenience Store,American Restaurant,Convention Center,Farmers Market
4,Downtown Columbus,39.962703,-82.997899,0.0,Coffee Shop,Hotel,Breakfast Spot,Theater,Italian Restaurant,Event Space,Latin American Restaurant,Salad Place,Business Service,Irish Pub


**Cluster 1**

In [145]:
cbus_merged.shape

(38, 14)

In [155]:
cbus_merged.loc[cbus_merged['Cluster Labels'] == 0, cbus_merged.columns[[0] + list(range(4, cbus_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Brewery District,Coffee Shop,Bakery,Bar,American Restaurant,Brewery,Theater,Beer Bar,Breakfast Spot,Restaurant,Italian Restaurant
2,Canal Winchester,Bank,American Restaurant,Pool,Brewery,Snack Place,Café,Bakery,Fast Food Restaurant,Historic Site,History Museum
3,Clintonville,Automotive Shop,Video Store,Sandwich Place,Bar,Fast Food Restaurant,Coffee Shop,Convenience Store,American Restaurant,Convention Center,Farmers Market
4,Downtown Columbus,Coffee Shop,Hotel,Breakfast Spot,Theater,Italian Restaurant,Event Space,Latin American Restaurant,Salad Place,Business Service,Irish Pub
5,Dublin,Pizza Place,Café,Italian Restaurant,Yoga Studio,Brewery,Ice Cream Shop,Steakhouse,Irish Pub,Southern / Soul Food Restaurant,Boutique
6,Eastmoor,Fast Food Restaurant,Chinese Restaurant,Discount Store,Mexican Restaurant,ATM,Gas Station,Convention Center,Cosmetics Shop,Cuban Restaurant,Dance Studio
7,Franklin Park,Park,Construction & Landscaping,Fried Chicken Joint,Baseball Field,Tennis Court,Athletics & Sports,Discount Store,Farmers Market,Fabric Shop,Event Space
8,Franklinton,Gas Station,Diner,Pizza Place,Food Service,Discount Store,Furniture / Home Store,Coffee Shop,Cuban Restaurant,Dance Studio,Deli / Bodega
9,Gahanna,Ice Cream Shop,Breakfast Spot,Bar,Clothing Store,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Cajun / Creole Restaurant,Memorial Site,Mexican Restaurant
10,German Village,Coffee Shop,American Restaurant,Bakery,German Restaurant,ATM,Salon / Barbershop,Beer Bar,Ice Cream Shop,Indie Theater,Brewery


**Cluster 2**

In [156]:
cbus_merged.loc[cbus_merged['Cluster Labels'] == 1, cbus_merged.columns[[0] + list(range(4, cbus_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bexley,Park,Yoga Studio,Department Store,Farmers Market,Fabric Shop,Event Space,Donut Shop,Dive Bar,Discount Store,Diner


**Cluster 3**

In [157]:
cbus_merged.loc[cbus_merged['Cluster Labels'] == 2, cbus_merged.columns[[0] + list(range(4, cbus_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Grove City,Wine Bar,Dive Bar,Yoga Studio,Department Store,Farmers Market,Fabric Shop,Event Space,Donut Shop,Discount Store,Diner


**Cluster 4**

In [158]:
cbus_merged.loc[cbus_merged['Cluster Labels'] == 3, cbus_merged.columns[[0] + list(range(4, cbus_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Linden,Discount Store,Caribbean Restaurant,Furniture / Home Store,Bank,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Space,Donut Shop
23,North Linden,Discount Store,Caribbean Restaurant,Furniture / Home Store,Bank,Yoga Studio,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Space,Donut Shop
25,Old Towne East,Fast Food Restaurant,Discount Store,Yoga Studio,Diner,Farmers Market,Fabric Shop,Event Space,Donut Shop,Dive Bar,Department Store


**Cluster 5**

In [159]:
cbus_merged.loc[cbus_merged['Cluster Labels'] == 4, cbus_merged.columns[[0] + list(range(4, cbus_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Minerva Park,Golf Course,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Fabric Shop,Event Space,Donut Shop,Dive Bar,Discount Store
